In [1]:
import csv
import random
import gensim
import numpy
import json
import pandas as pd
import jieba
import collections
from sklearn import metrics
from gensim.models import word2vec
from sklearn.cluster import KMeans
from openpyxl.workbook import Workbook
from collections import Counter

In [2]:
with open(r'E:\13万筛选三元组\triples-13w.json','r',encoding='utf-8')as fp:
    emotion_pairs = json.load(fp)
print(len(emotion_pairs))

130645


In [3]:
sxc = []
for i,y1 in enumerate(emotion_pairs):
    for j,y2 in enumerate(y1):
        if(len(y2['属性词'])!=0):
            if(y2['属性词'][0] not in sxc):
                sxc.append(y2['属性词'][0])

In [5]:
fw1 = open(r'E:\sxc.txt', 'w', encoding='utf-8')   
for i in sxc:
    fw1.write(str(i))
    fw1.write('\n')  #换行
fw1.close()

In [6]:
#获取评论语料
all_comments = ''
with open(r'E:\13万筛选三元组\content_13w.txt', encoding='utf-8') as f:
    for line in f:
        all_comments+=line.strip('\n')
        all_comments+='。'
f.close()
print(all_comments[:50])

每次来北京都住这里。位置方便。四环下来就是。停车位充足。而且院子里面很多好吃的。怎么能说是很多呢。简


In [7]:
jieba.setLogLevel(jieba.logging.INFO)
path_jieba = r"C:\Users\11874\Desktop\kmeans聚类数据及代码\dict_fliter_set.txt" #人工修改后的词典
[jieba.suggest_freq(line.strip(), True) for line in open(path_jieba,'r',encoding='utf8')]#strip默认空格或换行符 调节单个词语的词频，使其能（或不能）被分出来

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 8,
 1,
 8,
 2,
 4,
 1,
 1,
 128,
 1,
 1,
 1,
 24,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 32,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 9,
 1,
 4,
 1,
 1,
 4,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 10,
 56,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 62,
 4,
 1,
 1,
 1,
 1,
 1,
 1537,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 7,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 23,
 9,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 9,
 1,
 1,
 19,
 1,
 12,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 27,
 1,
 94,
 1,
 1,
 1,
 1,
 1,
 18,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 11,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 4,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,

In [8]:
#分词
split_words=list(jieba.cut(all_comments)) 

In [9]:
#读取停用词表
stopwords=[line.strip() for line in open(r'C:\Users\11874\Desktop\kmeans聚类数据及代码\词典\stopword.txt', 'r',encoding='utf-8',errors='ignore').readlines()]
filtered_corpus1=[word for word in split_words if word not in stopwords]  #去停用词
filtered_corpus = ' '.join(filtered_corpus1)

FileNotFoundError: [Errno 2] No such file or directory: 'E:\\词向量\\jieba_cut.txt'

In [10]:
#保存
fo = open(r'E:\词向量\jieba_cut.txt', 'w', encoding='utf-8')  #分词保存到txt文件中
fo.write(filtered_corpus)
fo.close()

In [11]:
#训练word2vec词向量模型
sentences =word2vec.Text8Corpus(r'E:\词向量\jieba_cut.txt')  #读取刚保存的分词文件
model = word2vec.Word2Vec(sentences, min_count=3 ,vector_size=200)  #生成词向量空间min_count:对字典做截断.词频少于min_count次数的单词会被丢弃掉 size：是指特征向量的维度
model.save(r"E:\词向量\word2vec.model")      # 保存

from gensim.test.utils import common_texts
from gensim.models import Word2Vec
model = Word2Vec.load(r"E:\词向量\word2vec.model")        # 读取,加载模型,模型可继续训练

In [12]:
wordvector = [] 
keys = []
for key in sxc:
    if(key in model.wv):
        keys.append(key)
        wordvector.append(model.wv[key])
print(len(keys))

1646


In [47]:
import numpy as np
X = []
X.append(model.wv.__getitem__(['空调']))
X.append(model.wv.__getitem__(['饺子']))
X.append(model.wv.__getitem__(['卫生间']))
X.append(model.wv.__getitem__(['健身房']))
X.append(model.wv.__getitem__(['地铁站']))
X.append(model.wv.__getitem__(['服务']))
X = np.mat(np.array(X,np.float64))

clf = KMeans(n_clusters=6,init=X)
s = clf.fit(wordvector)
labels = clf.labels_
print(metrics.silhouette_score(wordvector, s.labels_, metric='euclidean'))

0.50963116


D:\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [48]:
classCollects={0:[],1:[],2:[],3:[],4:[],5:[]}
for i in range(len(keys)):
    if labels[i] in list(classCollects.keys()):
        classCollects[labels[i]].append(list(keys)[i])
        
print('0类：',classCollects[0])
print('1类：',classCollects[1])
print('2类：',classCollects[2])
print('3类：',classCollects[3])
print('4类：',classCollects[4])
print('5类：',classCollects[5])

0类： ['隔音', '声音', '隔音效果', '空调', '休息', '噪音', '睡觉', '窗户', '晚上', '早上', '飞机', '隔壁', '说话']
1类： ['服务员', '维护', '前台接待人员', '插头', '价格浮动', '水质', '整洁度', '客房服务', '水量', '空调温度', '气味', '整整', '地下', '服务质量', '接待', '送餐', '餐品', '大堂人员', '素质', '整体服务', '静音效果', '住体验', '效率', '整体装修', '整体装潢', '程度', '房间装修风格', '酒吧', '整体来说', '风机', '夜景', '房间布局', '整体性价比', '入住手续', '地鐵站', '酒廊', '性價比', '反馈', '早餐质量', '总体性价比', '寝具', '床的软硬度', '床睡着', '入住感觉', '空气质量', '办事效率', '設施', '服務態度', '总体感受', '整体入住感觉', '舒适性', '视线', '屋顶', '整体体验', '视野', '网速', '实施', '大堂布置', '啤酒', '宾馆', '用餐环境', '质量', '接待人员', '总体体验', '食品', '房間', '服务生', '餐厅味道', '服务器', '速度', '大堂香味', '客人反馈', '前台效率', '维景', '住起来', '评价', '服務', '水压', '唐宫', '房型', '卫生状况', '停车场位置', '景观', '软件', '環境', '总体说', '床蛮', '客服态度', '早饭', '手续', '服务品质', '整体条件', '印象', '餐厅口味', '装潢', '服务环境', '总体来看', '早晨', '餐食', '空调声音', '保护', '信价比', '办理入住', '退房', '服务设施', '前台态度', '总体评价', '床具', '电视遥控器', '景色', '贵贵', '陈设', '入住环境', '客户体验', '早点', '坏境', '游泳', '酒店内部', '时间', '用户', '办理退房', '房间布置', '服务更', '客房人员', '适度', '房间大小', '办理速度', '早餐吃的', '总得来说'

In [49]:
f = open(r'E:\13w聚类结果\1.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects.items():
    writer.writerow(l)
f.close()

## 迭代1

In [50]:
sxc1 = []
f = open(r'E:\13w聚类结果\1.csv',encoding='gbk')
csv_read = csv.reader(f)
index = 0
for line in csv_read:
    index = index + 1
    if index ==  7:
        sxc1 = sxc1+line
f.close()
print(len(sxc1))
print(sxc1)

1321
['价格浮动', '客房服务', '性价比', '总体', '价位', '气味', '总体来说', '价格', '整体', '选择', '体验', '家人', '出差', '住宿', '房价', '价钱', '入住体验', '各方面', '家庭', '总体感觉', '标准', '住宿环境', '居住', '条件', '感觉', '整整', '优惠', '完美', '住宿体验', '高', '品牌', '整体感觉', '地下', '收费', '静音效果', '住体验', '效率', '程度', '整体来说', '风机', '夜景', '整体性价比', '入住手续', '酒廊', '性價比', '反馈', '总体性价比', '入住感觉', '空气质量', '办事效率', '設施', '服務態度', '总体感受', '整体入住感觉', '舒适性', '视线', '屋顶', '整体体验', '视野', '网速', '实施', '大堂布置', '啤酒', '宾馆', '用餐环境', '质量', '接待人员', '总体体验', '食品', '房間', '服务生', '餐厅味道', '服务器', '速度', '大堂香味', '客人反馈', '前台效率', '维景', '住起来', '评价', '服務', '水压', '唐宫', '房型', '卫生状况', '停车场位置', '景观', '软件', '環境', '总体说', '床蛮', '客服态度', '早饭', '手续', '服务品质', '整体条件', '印象', '餐厅口味', '装潢', '服务环境', '总体来看', '早晨', '餐食', '空调声音', '保护', '信价比', '办理入住', '退房', '服务设施', '前台态度', '总体评价', '床具', '电视遥控器', '景色', '贵贵', '陈设', '入住环境', '客户体验', '早点', '坏境', '游泳', '酒店内部', '时间', '用户', '办理退房', '房间布置', '服务更', '客房人员', '适度', '房间大小', '办理速度', '早餐吃的', '总得来说', '效果', '阳光', 'WIFI', '睡起来', '办理手续', '住宿条件', '地址', '硬件软件', '氛围', '洗澡间', '整体房间'

In [51]:
wordvector1 = [] 
keys1 = []
for key in sxc1:
    if(key in model.wv):
        keys1.append(key)
        wordvector1.append(model.wv[key])
print(len(keys1))

1320


In [55]:
import numpy as np
X = []
X.append(model.wv.__getitem__(['飘窗']))
X.append(model.wv.__getitem__(['热菜味道']))
X.append(model.wv.__getitem__(['洗浴间']))
X.append(model.wv.__getitem__(['楼梯']))
X.append(model.wv.__getitem__(['地理']))
X.append(model.wv.__getitem__(['客服态度']))
X = np.mat(np.array(X,np.float64))

clf = KMeans(n_clusters=6,init=X)
s = clf.fit(wordvector1)
labels = clf.labels_
print(metrics.silhouette_score(wordvector1, s.labels_, metric='euclidean'))

0.34889352


D:\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [56]:
classCollects1={0:[],1:[],2:[],3:[],4:[],5:[]}
for i in range(len(keys1)):
    if labels[i] in list(classCollects1.keys()):
        classCollects1[labels[i]].append(list(keys1)[i])
        
print('0类：',classCollects1[0])
print('1类：',classCollects1[1])
print('2类：',classCollects1[2])
print('3类：',classCollects1[3])
print('4类：',classCollects1[4])
print('5类：',classCollects1[5])

0类： ['地下', '收费', '反馈', '宾馆', '食品', '唐宫', '早饭', '早晨', '餐食', '早点', '游泳', '酒店内部', '住宿条件', '地址', '自助洗衣房', '吃饭逛街', '饮食', '看病', '住店', '就餐', '食物', '吃喝', '餐饮店', '用餐', '午餐', '东西', '早餐味道', '叫车', '赶飞机', '晚饭', '吃喝玩乐', '停车券', '地铁出行', '出入', '亲子', '细节设计', '洗衣', '总体来讲', '游玩', '公交都', '早餐店', '热菜', '点心', '商店', '接送机', '车子', '机场接送', '休闲', '早班', '出去玩', '反正', '烤鸭', '地处', '坐车', '周边小吃', '接机', '送机', '接送', '接送车', '门面', '奥体', '路线', '院子', '外出', '风味', '车站', '吃早餐', '专车', '商务人士', '赶时间', '换乘', '菜', '乘车', '机场大巴', '出发', '早班机', '汽车', '路过', '自驾', '导航', '绿化', '路边', '吃早饭', '饭馆', '出租', '坐飞机', '太多', '不远处', '附近吃的', '办事情', '老板', '粤菜', '大巴', '巴士', '散步', '平谷', '简餐', '健身', '中餐厅', '逛逛', '京味', '站台', '饺子', '花园', '全家', '游客', '昌平', '市场', '地道', '大栅栏', '地儿', '住一晚', '地区', '相对来说', '水果店', '路口']
1类： ['价格浮动', '整整', '静音效果', '住体验', '风机', '夜景', '整体性价比', '性價比', '总体性价比', '入住感觉', '空气质量', '办事效率', '总体感受', '整体入住感觉', '舒适性', '视线', '整体体验', '实施', '大堂布置', '用餐环境', '接待人员', '总体体验', '餐厅味道', '服务器', '大堂香味', '客人反馈', '前台效率', '维景', '停车场位置', '总体说', '床蛮', '客服态度', '服务

In [57]:
f = open(r'E:\13w聚类结果\2.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects1.items():
    writer.writerow(l)
f.close()

## 迭代2

In [58]:
sxc2 = []
f = open(r'E:\13w聚类结果\2.csv',encoding='gbk')
csv_read = csv.reader(f)
index = 0
for line in csv_read:
    index = index + 1
    if index ==  7:
        sxc2 = sxc2+line
f.close()
print(len(sxc2))
print(sxc2)

909
['收费', '整体入住', '总体', '总体来说', '整体', '选择', '体验', '家人', '住宿', '入住体验', '各方面', '家庭', '总体感觉', '标准', '住宿环境', '居住', '条件', '感觉', '体验感', '完美', '住宿体验', '高', '品牌', '整体感觉', '评价', '印象', '水平', '希望', '感受', '性价比', '价位', '朝向', '单独', '费用', '出差', '房价', '价钱', '价格', '东西', '休闲', '早班', '反正', '门面', '商务人士', '太多', '办事情', '老板', '京味', '住一晚', '相对来说', '房费', '情况', '携程', '提示', '押金', '发票', '外卖', '早', '差评', '修', '气质', '优惠', '间', '预期', '出色', '提高', '内容', '入住感受', '反馈', '价格浮动', '入住时间', '运气', '感覺', '離', '价', '程度', '整体来说', '整整', '住宿条件', '点', '住起来', '住店', '静音效果', '亲子', '住体验', '风机', '总体来讲', '夜景', '总体评价', '整体性价比', '性價比', '总体性价比', '入住感觉', '空气质量', '办事效率', '总体感受', '整体入住感觉', '舒适性', '视线', '整体体验', '实施', '大堂布置', '用餐环境', '接待人员', '总体体验', '餐厅味道', '服务器', '大堂香味', '客人反馈', '前台效率', '维景', '停车场位置', '总体说', '床蛮', '客服态度', '服务品质', '整体条件', '餐厅口味', '服务环境', '总体来看', '保护', '信价比', '服务设施', '床具', '电视遥控器', '贵贵', '客户体验', '坏境', '服务更', '客房人员', '适度', '办理速度', '早餐吃的', '总得来说', '硬件软件', '整体房间', '员工素质', '服务效率', '工作效率', '上网速度', '退房手续', '大堂工作人员', '地暖', '家店', '卫生管理',

In [59]:
wordvector2 = [] 
keys2 = []
for key in sxc2:
    if(key in model.wv):
        keys2.append(key)
        wordvector2.append(model.wv[key])
print(len(keys2))

908


In [71]:
import numpy as np
X = []
X.append(model.wv.__getitem__(['空调暖气']))
X.append(model.wv.__getitem__(['食材']))
X.append(model.wv.__getitem__(['洗澡的水']))
X.append(model.wv.__getitem__(['会议室']))
X.append(model.wv.__getitem__(['机场快轨']))
X.append(model.wv.__getitem__(['服务管理']))
X = np.mat(np.array(X,np.float64))

clf = KMeans(n_clusters=6,init=X)
s = clf.fit(wordvector2)
labels = clf.labels_
print(metrics.silhouette_score(wordvector2, s.labels_, metric='euclidean'))

0.407342


D:\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [72]:
classCollects2={0:[],1:[],2:[],3:[],4:[],5:[]}
for i in range(len(keys2)):
    if labels[i] in list(classCollects2.keys()):
        classCollects2[labels[i]].append(list(keys2)[i])
        
print('0类：',classCollects2[0])
print('1类：',classCollects2[1])
print('2类：',classCollects2[2])
print('3类：',classCollects2[3])
print('4类：',classCollects2[4])
print('5类：',classCollects2[5])

0类： ['价格浮动', '静音效果', '风机', '总体感受', '整体入住感觉', '视线', '大堂布置', '用餐环境', '服务器', '大堂香味', '客人反馈', '维景', '停车场位置', '总体说', '床蛮', '客服态度', '整体条件', '餐厅口味', '总体来看', '信价比', '床具', '贵贵', '坏境', '服务更', '客房人员', '适度', '整体房间', '服务效率', '工作效率', '上网速度', '大堂工作人员', '地暖', '家店', '卫生管理', '装修设施', '空调设置', '洗发水味道', '床软硬', '地铁线', '早餐环境', 'wifi信号', '保安人员', '棉织品', '吃饭出行', '早餐品类', '内设', '甲醛味道', '换了房型', '调调', '洁具', '再优惠', '吃饭选择', '早餐品质', '早餐供应', '三餐', '业务能力', '安全措施', '极简', '软硬度', '网络信号', '洗漱用品质量', '住着感觉', '整体布局', '服务生态度', '推广', '停車', '整体印象', 'Fi速度', '早餐感觉', '前台接待态度', '公差', '入住态度', '客户反馈', '价贵', '装修效果', '京承', '出品', '床垫软硬', '周边商圈', '电视机尺寸', '素养', '卫生间灯光', '人品', '出水量', '排气', '床睡起来', '早餐口味', '入住效率', '整体空间', '补餐', '卫生房间', '标间床', '客服人员态度', '总体情况', '客观评价', '吃起来', '停车库', '早餐的质量', '无线信号', '总体水平', '办理时间', '空调制冷效果', '房间装饰', '装修设备', '室外环境', '淋浴器', '加餐', '吃喝玩', '用品质量', '总台服务员', '装修布置', '大堂味道', '新品', '喷头水', '餐桌', '大堂接待', '房间物品', '卫生间玻璃', '床品睡着', '大堂感觉', '公交总站', '床垫子', '热菜味道', '床品质量', '便利性', 'WIFI信号', '装修品质', '床的软硬', '整体感受', '员工态度', '整体而言

In [73]:
f = open(r'E:\13w聚类结果\3.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects2.items():
    writer.writerow(l)
f.close()

## 迭代3

In [74]:
sxc3 = []
f = open(r'E:\13w聚类结果\3.csv',encoding='gbk')
csv_read = csv.reader(f)
index = 0
for line in csv_read:
    index = index + 1
    if index ==  8:
        sxc3 = sxc3+line
f.close()
print(len(sxc3))
print(sxc3)

682
['价格浮动', '静音效果', '风机', '总体感受', '整体入住感觉', '视线', '大堂布置', '用餐环境', '服务器', '大堂香味', '客人反馈', '维景', '停车场位置', '总体说', '床蛮', '客服态度', '整体条件', '餐厅口味', '总体来看', '信价比', '床具', '贵贵', '坏境', '服务更', '客房人员', '适度', '整体房间', '服务效率', '工作效率', '上网速度', '大堂工作人员', '地暖', '家店', '卫生管理', '装修设施', '空调设置', '洗发水味道', '床软硬', '地铁线', '早餐环境', 'wifi信号', '保安人员', '棉织品', '吃饭出行', '早餐品类', '内设', '甲醛味道', '换了房型', '调调', '洁具', '再优惠', '吃饭选择', '早餐品质', '早餐供应', '三餐', '业务能力', '安全措施', '极简', '软硬度', '网络信号', '洗漱用品质量', '住着感觉', '整体布局', '服务生态度', '推广', '停車', '整体印象', 'Fi速度', '早餐感觉', '前台接待态度', '公差', '入住态度', '客户反馈', '价贵', '装修效果', '京承', '出品', '床垫软硬', '周边商圈', '电视机尺寸', '素养', '卫生间灯光', '人品', '出水量', '排气', '床睡起来', '早餐口味', '入住效率', '整体空间', '补餐', '卫生房间', '标间床', '客服人员态度', '总体情况', '客观评价', '吃起来', '停车库', '早餐的质量', '无线信号', '总体水平', '办理时间', '空调制冷效果', '房间装饰', '装修设备', '室外环境', '淋浴器', '加餐', '吃喝玩', '用品质量', '总台服务员', '装修布置', '大堂味道', '新品', '喷头水', '餐桌', '大堂接待', '房间物品', '卫生间玻璃', '床品睡着', '大堂感觉', '公交总站', '床垫子', '热菜味道', '床品质量', '便利性', 'WIFI信号', '装修品质', '床的软硬', '整体感受', '员工态度', '整体而言

In [75]:
wordvector3 = [] 
keys3 = []
for key in sxc3:
    if(key in model.wv):
        keys3.append(key)
        wordvector3.append(model.wv[key])
print(len(keys3))

681


In [76]:
import numpy as np
X = []
X.append(model.wv.__getitem__(['空调暖气']))
X.append(model.wv.__getitem__(['食材']))
X.append(model.wv.__getitem__(['洗澡的水']))
X.append(model.wv.__getitem__(['会议室']))
X.append(model.wv.__getitem__(['机场快轨']))
X.append(model.wv.__getitem__(['服务管理']))
X = np.mat(np.array(X,np.float64))

clf = KMeans(n_clusters=6,init=X)
s = clf.fit(wordvector3)
labels = clf.labels_
print(metrics.silhouette_score(wordvector3, s.labels_, metric='euclidean'))

0.18211803


D:\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [78]:
classCollects3={0:[],1:[],2:[],3:[],4:[],5:[]}
for i in range(len(keys3)):
    if labels[i] in list(classCollects3.keys()):
        classCollects3[labels[i]].append(list(keys3)[i])
        
print('0类：',classCollects3[0])
print('1类：',classCollects3[1])
print('2类：',classCollects3[2])
print('3类：',classCollects3[3])
print('4类：',classCollects3[4])
print('5类：',classCollects3[5])

0类： ['价格浮动', '静音效果', '服务器', '大堂香味', '客人反馈', '停车场位置', '总体说', '床蛮', '餐厅口味', '信价比', '贵贵', '适度', '服务效率', '上网速度', '家店', '卫生管理', '空调设置', '洗发水味道', 'wifi信号', '保安人员', '内设', '换了房型', '早餐供应', '安全措施', '软硬度', '网络信号', '住着感觉', '推广', '入住态度', '客户反馈', '价贵', '装修效果', '京承', '周边商圈', '电视机尺寸', '卫生间灯光', '早餐口味', '整体空间', '卫生房间', '客服人员态度', '总体情况', '停车库', '早餐的质量', '无线信号', '总体水平', '装修设备', '室外环境', '淋浴器', '总台服务员', '装修布置', '大堂味道', '新品', '大堂接待', '房间物品', '床品睡着', '装修品质', '床的软硬', '整体而言', 'Fi信号', '欧洲', '套件', '卫生装修', '室内设计', '难度', '安保措施', '整体水平', '风水', '整体效果', '交通条件', '总体入住感觉', '业界', '柜台人员', '整体配备', '枕头床', '卫生间布局', '干净度', '接送机师傅', '卫生到', '床特', '暖水', '整理感觉', '设备感觉', '卫生均', '方形', '硬软件', '住着环境', '分工', '自助洗衣机', '自助洗衣服', '床软硬度', '整体体验感', '热水温度', '隔音设施', '附近吃的店', '饭味', '平面', '房间装修设计', '去西站', '卫生间味道', '价廉', '用具质量', '开间', '地位', '坐早班机', '浴室玻璃', '服务接待', '早餐面条', '升级房', '房间格局布置', '网络覆盖', '安全设施', '住置', '灯具', '入住感', '隔音措施', '客用品', '茶品', '食店', '整体设计风格', '吃饭地', '倍儿', 'WiFi信号', '卫生间设计', '旱餐', '服务速度', '入住体验完', '空间体验', '喷水', '处理态度', '窗景', '入住条

In [79]:
f = open(r'E:\13w聚类结果\4.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects3.items():
    writer.writerow(l)
f.close()

## 迭代4

In [80]:
sxc4 = []
f = open(r'E:\13w聚类结果\4.csv',encoding='gbk')
csv_read = csv.reader(f)
index = 0
for line in csv_read:
    index = index + 1
    if index ==  8:
        sxc4 = sxc4+line
f.close()
print(len(sxc4))
print(sxc4)

359
['价格浮动', '性', '静音效果', '服务器', '大堂香味', '客人反馈', '停车场位置', '总体说', '床蛮', '餐厅口味', '價格', '信价比', '贵贵', '适度', '服务效率', '上网速度', '家店', '卫生管理', '空调设置', '洗发水味道', 'wifi信号', '保安人员', '内设', '整潔', '换了房型', '早餐供应', '安全措施', '软硬度', '网络信号', '住着感觉', '推广', '入住态度', '客户反馈', '价贵', '装修效果', '京承', '周边商圈', '电视机尺寸', '卫生间灯光', '早餐口味', '整体空间', '卫生房间', '客服人员态度', '总体情况', '停车库', '早餐的质量', '无线信号', '总体水平', '装修设备', '室外环境', '淋浴器', '总台服务员', '装修布置', '大堂味道', '新品', '大堂接待', '房间物品', '床品睡着', '装修品质', '床的软硬', '整体而言', 'Fi信号', '欧洲', '套件', '卫生装修', '室内设计', '难度', '安保措施', '整体水平', '风水', '整体效果', '交通条件', '总体入住感觉', '业界', '柜台人员', '整体配备', '枕头床', '卫生间布局', '干净度', '接送机师傅', '卫生到', '床特', '暖水', '整理感觉', '设备感觉', '卫生均', '方形', '硬软件', '住着环境', '分工', '自助洗衣机', '自助洗衣服', '床软硬度', '整体体验感', '热水温度', '隔音设施', '附近吃的店', '饭味', '平面', '房间装修设计', '去西站', '卫生间味道', '价廉', '用具质量', '开间', '地位', '坐早班机', '浴室玻璃', '服务接待', '早餐面条', '升级房', '房间格局布置', '网络覆盖', '安全设施', '住置', '灯具', '入住感', '隔音措施', '客用品', '茶品', '食店', '整体设计风格', '吃饭地', '倍儿', 'WiFi信号', '卫生间设计', '旱餐', '服务速度', '入住体验完', '空间体验', '喷水', '

In [81]:
wordvector4 = [] 
keys4 = []
for key in sxc4:
    if(key in model.wv):
        keys4.append(key)
        wordvector4.append(model.wv[key])
print(len(keys4))

358


In [82]:
import numpy as np
X = []
X.append(model.wv.__getitem__(['空调暖气']))
X.append(model.wv.__getitem__(['食材']))
X.append(model.wv.__getitem__(['洗澡的水']))
X.append(model.wv.__getitem__(['会议室']))
X.append(model.wv.__getitem__(['机场快轨']))
X.append(model.wv.__getitem__(['服务管理']))
X = np.mat(np.array(X,np.float64))

clf = KMeans(n_clusters=6,init=X)
s = clf.fit(wordvector4)
labels = clf.labels_
print(metrics.silhouette_score(wordvector4, s.labels_, metric='euclidean'))

0.08310466


D:\anaconda\lib\site-packages\sklearn\cluster\_kmeans.py:984: RuntimeWarning: Explicit initial center position passed: performing only one init in KMeans instead of n_init=10.
  self._check_params(X)


In [83]:
classCollects4={0:[],1:[],2:[],3:[],4:[],5:[]}
for i in range(len(keys4)):
    if labels[i] in list(classCollects4.keys()):
        classCollects4[labels[i]].append(list(keys4)[i])
        
print('0类：',classCollects4[0])
print('1类：',classCollects4[1])
print('2类：',classCollects4[2])
print('3类：',classCollects4[3])
print('4类：',classCollects4[4])
print('5类：',classCollects4[5])

0类： ['服务器', '总体说', '贵贵', '适度', '保安人员', '换了房型', '住着感觉', '推广', '卫生间灯光', '早餐口味', '卫生房间', '总体情况', '停车库', '总体水平', '装修设备', '室外环境', '总台服务员', '大堂接待', '装修品质', '床的软硬', '欧洲', '卫生装修', '安保措施', '总体入住感觉', '枕头床', '卫生间布局', '干净度', '接送机师傅', '卫生到', '床特', '暖水', '整理感觉', '设备感觉', '卫生均', '硬软件', '住着环境', '整体体验感', '热水温度', '隔音设施', '附近吃的店', '去西站', '价廉', '用具质量', '地位', '坐早班机', '早餐面条', '升级房', '网络覆盖', '安全设施', '住置', '入住感', '茶品', '食店', '整体设计风格', '倍儿', 'WiFi信号', '入住体验完', '房间玻璃', '牙刷质量', '设施硬件', '卫生间装修', '价算', '清洁度', 'WIFI速度', '格局布置', '茶座', '水流量', '服务台态度', '房间格调', '房务', '总体来说体验', '床睡觉', '卫生间条件', '综合性价比', '地角', '服好', '安全保障', '隔音的话', '无线网速度', '空调声', '吃行', '保洁服务', '房间质量', '消毒措施', '床软', '被服', '防控措施', '入室', '超市药店', '水床', '综合体验', '换算', '入住流程', '窗帘儿', '厕所味道', '整体算', '中餐味道', '光照', '服务配套', '叫餐', '前台服务意识', '餐饮配套', '机场出行', '浴室水', '饮食店', '洗漱产品', '好评率', '整体性', '装修档次', '餐厅可', '装修味儿', '客观的说', '安全门', '自动化设备', '方间', '推举', '空调暖气', '装修声音', '大堂时', '宾馆内', '外置', '园景', '茶台', '车流量', '服务员效率', '居住条件', '价比', '酒店设计感', '性能', '窗户密封', '超市等', '渗水', '住房条件

In [84]:
f = open(r'E:\13w聚类结果\5.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects4.items():
    writer.writerow(l)
f.close()

In [110]:
import csv
path = r'E:\13w聚类结果\ordered1.csv'
with open(path, 'r', encoding = 'gbk') as csvfile:
    reader = csv.reader(csvfile)
    rows = [row for row in reader]
    print(len(rows))

7


In [31]:
def is_exist_by_double_list(target, element):
    """
    判断二维列表中是否存在某个元素
    :param target:目标列表
    :param element:需要判断的元素
    :return:bool类型，是否存在
    """
    for line in target:
        if element in line:
            return True
    return False

def return_index_in_double_list(target, element):
    if(is_exist_by_double_list(target, element)==True):
        for i,k1 in enumerate(target):
            for j,k2 in enumerate(k1):
                if(element==k2):
                    return i,j
    else:
        return False

In [111]:
#初始化属性频次列表
fre_att = []
for i in range(len(rows)):
    fre_att.append([])
for i in range(len(fre_att)):
    for j in range(len(rows[i])):
        fre_att[i].append(0)

In [112]:

for i,y1 in enumerate(emotion_pairs):
    is_exist = [0]*len(rows)
    for j,y2 in enumerate(y1):
        if(len(y2['属性词'])!=0):#有词
            if(return_index_in_double_list(rows,y2['属性词'][0])!=False):#[0]去符号，留汉字
                index_att = return_index_in_double_list(rows,y2['属性词'][0])#词位于i,j
                if(is_exist[index_att[0]] == 0):
                    fre_att[index_att[0]][index_att[1]]+=1
                    is_exist[index_att[0]] = 1

In [113]:
fre_comment = []
for i in range(len(rows)):
    fre_comment.append(0)
    
for i,y1 in enumerate(emotion_pairs):
    is_exist = [0]*len(rows)
    for j,y2 in enumerate(y1):#y2每条评论里所有词          
            if(len(y2['属性词'])!=0):
                if(return_index_in_double_list(rows,y2['属性词'][0])!=False): #rows类    y3评论中一套词组
                    index_att = return_index_in_double_list(rows,y2['属性词'][0])  #i,j
                    if(is_exist[index_att[0]] == 0):
                        fre_comment[index_att[0]]+=1
                        is_exist[index_att[0]] = 1
print(fre_comment) #每一类出现的次数

of = list(np.argsort(np.argsort(fre_comment)))#对属性进行排序并写入文件

#对属性进行排序并写入文件
import numpy as np
fre_att2 = []
for i in fre_att:
    fre_att2.append([])
#print(fre_att2)
for i in range(len(fre_att)):
    fre_att2[i] = np.argsort(np.argsort(fre_att[i]))#从小到大排列，提取其对应的索引
    fre_att2[i] = list(fre_att2[i])
#print(fre_att2)#索引

sort = []
for i in fre_att2:
    sort.append([])
#print(sort)
for x in range(len(fre_att2)):
    for i in range(len(rows[x])):
        sort[x].append(rows[x][fre_att2[x].index(i)])#.index()查找位置
    sort[x] = list(reversed(sort[x]))#逆序排列

for x in range(len(sort)):
    fw2 = open(r'E:\13w聚类结果\ordered2.txt', 'a+', encoding='utf-8')     #a+,逐行写入
    fw2.write(str(x+1))     #写入序号，方便排序
    fw2.write(',')
    for i in sort[of.index(len(rows)-1-x)]:    ####i-x中的i等于聚类簇数-1
        fw2.write(str(i))
        fw2.write(',')
    fw2.write('\n')
    fw2.close()

[47790, 44293, 43086, 34276, 18681, 14736, 1975]


In [114]:
import linecache as lc 
classCollects={}
for i in range(7):
    line = lc.getline(r'E:\13w聚类结果\ordered2.txt', i+1)
    line = line.strip('\n')
    line = line.split(",")
    classCollects[i] = line

In [115]:
#结果保存，需要手动修改编码utf-8
f = open(r'E:\13w聚类结果\ordered2.csv','w',newline='')
writer = csv.writer(f)
for key,l in classCollects.items():
    writer.writerow(l)#将一个列表全部写入csv的同一行
f.close()